# IPS Experiment

## Metrics

In [ ]:
import numpy
import sys

In [ ]:
class Jitter:
    def __init__(self, cut_off, num_users, num_items):
        self.jitter = 1e-7 * numpy.random.standard_normal((num_users, num_items))
        discountParams = 2.0 + numpy.array(range(num_items), dtype = numpy.longdouble)
        self.discountParams = numpy.reciprocal(numpy.log2(discountParams))
        self.cutOff = min(cut_off, num_items)
        self.discountParams[self.cutOff:] = 0.0

        print ("Jitter.init: [DBG]\t (NumUsers, NumItems)"), num_users, num_items, ("\t Sum DiscountFactors"),\
                self.discountParams.sum(dtype = numpy.longdouble), ("\t [Requested/Set] Cut-off:"), \
                cut_off, self.cutOff

    def rank(self, predicted_matrix):
        transformedPredictions = -numpy.ma.add(predicted_matrix, self.jitter)
        sortedPredictions = numpy.ma.argsort(transformedPredictions, axis = 1)
        return sortedPredictions

In [ ]:
dcgJitter = None

In [ ]:
def SET_PROPENSITIES(observed_ratings, inverse_propensities, verbose = False):
    numObservations = numpy.ma.count(observed_ratings)
    numUsers, numItems = numpy.shape(observed_ratings)
    scale = numUsers * numItems
    inversePropensities = None
    if inverse_propensities is None:
        inversePropensities = numpy.ones((numUsers, numItems), dtype = numpy.longdouble) * scale /\
                            numObservations
    else:
        inversePropensities = numpy.array(inverse_propensities, dtype = numpy.longdouble, copy = True)

    inversePropensities = numpy.ma.array(inversePropensities, dtype = numpy.longdouble, copy = False, 
                            mask = numpy.ma.getmask(observed_ratings), fill_value = 0, hard_mask = True)
 
    if verbose:
        print ("Metrics.SET_PROPENSITIES: [LOG]\t NumUsers, NumItems, NumObservations"), \
            numUsers, numItems, numObservations
        print ("Metrics.SET_PROPENSITIES: [DBG]\t Sum of observed inverse propensities "), \
            numpy.ma.sum(inversePropensities, dtype = numpy.longdouble), \
            ("(=? NumUsers * NumItems)"), numUsers * numItems

    return inversePropensities

In [ ]:
def ITEMWISE_METRICS(observed_ratings, predicted_ratings, inverse_propensities, verbose, mode = 'MSE'):
    delta = numpy.ma.subtract(predicted_ratings, observed_ratings)
    rectifiedDelta = None
    if mode == 'MSE':
        rectifiedDelta = numpy.square(delta)
    elif mode == 'MAE':
        rectifiedDelta = numpy.ma.abs(delta)
    else:
        print ("Metrics.ITEMWISE_METRICS: [ERR]\t Unrecognized itemwise metric "), mode
        sys.exit(0)

    inversePropensities = SET_PROPENSITIES(observed_ratings, inverse_propensities, verbose)

    numUsers, numItems = numpy.shape(observed_ratings)
    scale = numUsers * numItems

    observedError = numpy.ma.multiply(rectifiedDelta, inversePropensities)
    cumulativeError = numpy.ma.sum(observedError, dtype = numpy.longdouble)
    vanillaMetric = cumulativeError / scale
    
    globalNormalizer = numpy.ma.sum(inversePropensities, dtype = numpy.longdouble)
    selfNormalizedMetric = cumulativeError / globalNormalizer
    
    perUserNormalizer = numpy.ma.sum(inversePropensities, axis = 1, dtype = numpy.longdouble)
    perUserNormalizer = numpy.ma.masked_less_equal(perUserNormalizer, 0.0, copy = False)

    perUserError = numpy.ma.sum(observedError, axis = 1, dtype = numpy.longdouble)
    perUserEstimate = numpy.ma.divide(perUserError, perUserNormalizer)
    userNormalizedMetric = numpy.ma.sum(perUserEstimate, dtype = numpy.longdouble) / numUsers

    perItemNormalizer = numpy.ma.sum(inversePropensities, axis = 0, dtype = numpy.longdouble)
    perItemNormalizer = numpy.ma.masked_less_equal(perItemNormalizer, 0.0, copy = False)

    perItemError = numpy.ma.sum(observedError, axis = 0, dtype = numpy.longdouble)
    perItemEstimate = numpy.ma.divide(perItemError, perItemNormalizer)
    itemNormalizedMetric = numpy.ma.sum(perItemEstimate, dtype = numpy.longdouble) / numItems
   
    if verbose:
        print ("Metrics.ITEMWISE_METRICS: [LOG]\t Vanilla, SelfNormalized, UserNormalized, ItemNormalized"), \
            vanillaMetric, selfNormalizedMetric, userNormalizedMetric, itemNormalizedMetric

    return vanillaMetric, selfNormalizedMetric, userNormalizedMetric, itemNormalizedMetric

In [ ]:
def MSE(observed_ratings, predicted_ratings, inverse_propensities, verbose = False):
    return ITEMWISE_METRICS(observed_ratings, predicted_ratings, inverse_propensities, verbose, mode = 'MSE')

In [ ]:
def MAE(observed_ratings, predicted_ratings, inverse_propensities, verbose = False):
    return ITEMWISE_METRICS(observed_ratings, predicted_ratings, inverse_propensities, verbose, mode = 'MAE')

In [ ]:
def DCG(observed_ratings, predicted_ratings, inverse_propensities, cut_off = 50, verbose = False):
    global dcgJitter
    numUsers, numItems = numpy.shape(observed_ratings)
    scale = numUsers * numItems

    if dcgJitter is None or dcgJitter.cutOff != cut_off:
        dcgJitter = Jitter(cut_off, numUsers, numItems)
 
    inversePropensities = SET_PROPENSITIES(observed_ratings, inverse_propensities, verbose)
    
    predictedRankings = dcgJitter.rank(predicted_ratings)
    weightedGain = numpy.ma.multiply(observed_ratings, inversePropensities)
 
    perUserNormalizer = numpy.ma.sum(inversePropensities, axis = 1, dtype = numpy.longdouble)
    perUserNormalizer = numpy.ma.masked_less_equal(perUserNormalizer, 0.0, copy = False)

    staticIndices = numpy.ogrid[0:numUsers, 0:numItems]
    rankedGains = weightedGain[staticIndices[0], predictedRankings]
    perUserDCG = numpy.ma.dot(rankedGains, dcgJitter.discountParams)

    dcgValue = numpy.ma.sum(perUserDCG, dtype = numpy.longdouble) / numUsers
    snDCGValue = dcgValue * scale / numpy.ma.sum(inversePropensities, dtype = numpy.longdouble)

    perUserNormalizedEstimates = numpy.ma.divide(perUserDCG, perUserNormalizer)
    uDCGValue = numItems * numpy.ma.sum(perUserNormalizedEstimates, dtype = numpy.longdouble) / numUsers
    
    if verbose:
        print ("Metrics.DCG: [LOG]\t DCG, SN-DCG, UN-DCG, IN-DCG"), dcgValue, snDCGValue, uDCGValue, 0.0
    return dcgValue, snDCGValue, uDCGValue, 0.0

In [ ]:
def CG(observed_ratings, selected_items, inverse_propensities, verbose = False):
    inversePropensities = SET_PROPENSITIES(observed_ratings, inverse_propensities, verbose)

    clippedSelections = numpy.clip(selected_items, 0, 1)
    weightedGain = numpy.ma.multiply(observed_ratings, inversePropensities)
    cumulativeGain = numpy.ma.multiply(weightedGain, clippedSelections)
    
    numUsers, numItems = numpy.shape(observed_ratings)
    scale = numUsers * numItems

    globalGain = numpy.ma.sum(cumulativeGain, dtype = numpy.longdouble)
    globalNormalizer = numpy.ma.sum(inversePropensities, dtype = numpy.longdouble)

    cg = globalGain / numUsers
    snCG = numItems * globalGain / globalNormalizer

    perUserNormalizer = numpy.ma.sum(inversePropensities, axis = 1, dtype = numpy.longdouble)
    perUserNormalizer = numpy.ma.masked_less_equal(perUserNormalizer, 0.0, copy = False)

    perUserGain = numpy.ma.sum(cumulativeGain, axis = 1, dtype = numpy.longdouble)
    perUserEstimate = numpy.ma.divide(perUserGain, perUserNormalizer)
    unCG = numItems * numpy.ma.sum(perUserEstimate, dtype = numpy.longdouble) / numUsers

    perItemNormalizer = numpy.ma.sum(inversePropensities, axis = 0, dtype = numpy.longdouble)
    perItemNormalizer = numpy.ma.masked_less_equal(perItemNormalizer, 0.0, copy = False)

    perItemGain = numpy.ma.sum(cumulativeGain, axis = 0, dtype = numpy.longdouble)
    perItemEstimate = numpy.ma.divide(perItemGain, perItemNormalizer)
    inCG = numpy.ma.sum(perItemEstimate, dtype = numpy.longdouble)
       
    if verbose:
        print ("Metrics.CG: [LOG]\t CG, SN-CG, UN-CG, IN-CG"), cg, snCG, unCG, inCG
    return cg, snCG, unCG, inCG

In [ ]:
if __name__ == "__main__":
    shape = (5,3)
    a = numpy.random.randint(0,5, size=shape)
    b = numpy.random.randint(0,5, size=shape)
    
    print ("[MAIN]\t True ratings:")
    print (a)
    print ("[MAIN]\t Predicted ratings:")
    print (b)
    
    inversePropensities = numpy.random.random(shape)
    print ("[MAIN]\t Propensities:")
    print (inversePropensities)
    obs = numpy.random.random(shape)
    obs = obs < inversePropensities
    inversePropensities = numpy.reciprocal(inversePropensities)
    print ("[MAIN]\t Inverse Propensities:")
    print (inversePropensities)

    print ("[MAIN]\t Observations:")
    print (obs)
    
    observed_a = numpy.ma.array(a, dtype = numpy.longdouble, copy = True, 
                            mask = numpy.logical_not(obs), fill_value = 0, hard_mask = True)
     
    print ("[MAIN]\t MSE: Vanilla, SN, UN, IN:"),
    MSE(observed_a, b, inversePropensities, verbose = True)
    print ("[MAIN]\t MAE: Vanilla, SN, UN, IN:")
    MAE(observed_a, b, inversePropensities, verbose = True)
    print ("[MAIN]\t DCG: Vanilla, SN, UN, IN:")
    DCG(observed_a, b, inversePropensities, cut_off = 50, verbose = True)
    
    print ("[MAIN]\t CG: Vanilla, SN, UN, IN:")
    CG(observed_a, b, inversePropensities, verbose = True)

## MF

In [ ]:
# import numpy
import scipy.optimize
# import sys
# import Metrics

In [ ]:
def PREDICTED_SCORES(user_vectors, item_vectors, user_biases, item_biases, global_bias, use_bias = True):
    rawScores = numpy.dot(user_vectors, item_vectors.T)
    if use_bias:
        biasedScores = rawScores + user_biases[:,None] + item_biases[None,:] + global_bias
        return biasedScores
    else:
        return rawScores

In [ ]:
def GENERATE_MATRIX(observed_ratings, inverse_propensities, l2_regularization, num_dimensions, normalization,
        bias_mode = 'Regularized', mode = 'MSE', start_vec = None, verbose = False):

    metricMode = None
    if mode == 'MSE':
        metricMode = 1
    elif mode == 'MAE':
        metricMode = 2
    else:
        print("MF.GENERATE_MATRIX: [ERR]\t Metric not supported:", mode)
        sys.exit(0)

#     inversePropensities = Metrics.SET_PROPENSITIES(observed_ratings, inverse_propensities, False)
    inversePropensities = SET_PROPENSITIES(observed_ratings, inverse_propensities, False)

    numUsers, numItems = numpy.shape(observed_ratings)
    scale = numUsers * numItems
    numObservations = numpy.ma.count(observed_ratings)

    perUserNormalizer = numpy.ma.sum(inversePropensities, axis = 1, dtype = numpy.longdouble)
    perUserNormalizer = numpy.ma.masked_less_equal(perUserNormalizer, 0.0, copy = False)

    perItemNormalizer = numpy.ma.sum(inversePropensities, axis = 0, dtype = numpy.longdouble)
    perItemNormalizer = numpy.ma.masked_less_equal(perItemNormalizer, 0.0, copy = False)

    globalNormalizer = numpy.ma.sum(inversePropensities, dtype = numpy.longdouble)

    normalizedPropensities = None
    if normalization == 'Vanilla':
        normalizedPropensities = inversePropensities
    elif normalization == 'SelfNormalized':
        normalizedPropensities = scale * numpy.ma.divide(inversePropensities, globalNormalizer)
    elif normalization == 'UserNormalized':
        normalizedPropensities = numItems * numpy.ma.divide(inversePropensities, perUserNormalizer[:, None])
    elif normalization == 'ItemNormalized':
        normalizedPropensities = numUsers * numpy.ma.divide(inversePropensities, perItemNormalizer[None, :])
    else:
        print("MF.GENERATE_MATRIX: [ERR]\t Normalization not supported:", normalization)
        sys.exit(0)
    
    useBias = None
    regularizeBias = None
    if bias_mode == 'None':
        useBias = False
        regularizeBias = False
    elif bias_mode == 'Regularized':
        useBias = True
        regularizeBias = True
    elif bias_mode == 'Free':
        useBias = True
        regularizeBias = False
    else:
        print("MF.GENERATE_MATRIX: [ERR]\t Bias mode not supported:", bias_mode)
        sys.exit(0)

    if verbose:
        print("MF.GENERATE_MATRIX: [LOG]\t Lamda:", l2_regularization, "\t NumDims:", num_dimensions,\
            "\t Normalization:", normalization, "\t Metric:", mode, "\t BiasMode:", bias_mode)

    normalizedPropensities = numpy.ma.filled(normalizedPropensities, 0.0)
    observedRatings = numpy.ma.filled(observed_ratings, 0)
    
    def Mat2Vec(user_vectors, item_vectors, user_biases, item_biases, global_bias):
        allUserParams = numpy.concatenate((user_vectors, user_biases[:,None]), axis = 1)
        allItemParams = numpy.concatenate((item_vectors, item_biases[:,None]), axis = 1)
        
        allParams = numpy.concatenate((allUserParams, allItemParams), axis = 0)
        paramVector = numpy.reshape(allParams, (numUsers + numItems)*(num_dimensions + 1))
        paramVector = numpy.concatenate((paramVector, [global_bias]))
        return paramVector.astype(numpy.float)
        
    def Vec2Mat(paramVector):
        globalBias = paramVector[-1]
        remainingParams = paramVector[:-1]
        allParams = numpy.reshape(remainingParams, (numUsers + numItems, num_dimensions + 1))
        allUserParams = allParams[0:numUsers,:]
        allItemParams = allParams[numUsers:, :]
        
        userVectors = (allUserParams[:,0:-1]).astype(numpy.longdouble)
        userBiases = (allUserParams[:,-1]).astype(numpy.longdouble)
        
        itemVectors = (allItemParams[:,0:-1]).astype(numpy.longdouble)
        itemBiases = (allItemParams[:,-1]).astype(numpy.longdouble)
        return userVectors, itemVectors, userBiases, itemBiases, globalBias
    
    def Objective(paramVector):
        userVectors, itemVectors, userBiases, itemBiases, globalBias = Vec2Mat(paramVector)
        biasedScores = PREDICTED_SCORES(userVectors, itemVectors, userBiases, itemBiases, globalBias, useBias)

        delta = numpy.subtract(biasedScores, observedRatings)
        loss = None
        if metricMode == 1:
            loss = numpy.square(delta)
        elif metricMode == 2:
            loss = numpy.abs(delta)
        else:
            sys.exit(0)

        weightedLoss = numpy.multiply(loss, normalizedPropensities)
        objective = numpy.sum(weightedLoss, dtype = numpy.longdouble)

        gradientMultiplier = None
        if metricMode == 1:
            gradientMultiplier = numpy.multiply(normalizedPropensities, 2 * delta)
        elif metricMode == 2:
            gradientMultiplier = numpy.zeros(numpy.shape(delta), dtype = numpy.int)
            gradientMultiplier[delta > 0] = 1
            gradientMultiplier[delta < 0] = -1
            gradientMultiplier = numpy.multiply(normalizedPropensities, gradientMultiplier)
        else:
            sys.exit(0)

        userVGradient = numpy.dot(gradientMultiplier, itemVectors)
        itemVGradient = numpy.dot(gradientMultiplier.T, userVectors)

        userBGradient = None
        itemBGradient = None
        globalBGradient = None
        if useBias:
            userBGradient = numpy.sum(gradientMultiplier, axis = 1, dtype = numpy.longdouble)
            itemBGradient = numpy.sum(gradientMultiplier, axis = 0, dtype = numpy.longdouble)
            globalBGradient = numpy.sum(gradientMultiplier, dtype = numpy.longdouble)
        else:
            userBGradient = numpy.zeros(numpy.shape(userBiases), dtype = numpy.longdouble)
            itemBGradient = numpy.zeros(numpy.shape(itemBiases), dtype = numpy.longdouble)
            globalBGradient = 0.0

        if l2_regularization > 0:
            scaledPenalty = 1.0 * l2_regularization * scale / (numUsers + numItems)
            if regularizeBias:
                scaledPenalty /= (num_dimensions + 1)
            else:
                scaledPenalty /= num_dimensions

            userVGradient += 2 * scaledPenalty * userVectors
            itemVGradient += 2 * scaledPenalty * itemVectors
          
            objective += scaledPenalty * numpy.sum(numpy.square(userVectors), dtype = numpy.longdouble)
            objective += scaledPenalty * numpy.sum(numpy.square(itemVectors), dtype = numpy.longdouble)
 
            if regularizeBias:
                userBGradient += 2 * scaledPenalty * userBiases
                itemBGradient += 2 * scaledPenalty * itemBiases
                globalBGradient += 2 * scaledPenalty * globalBias
                objective += scaledPenalty * numpy.sum(numpy.square(userBiases), dtype = numpy.longdouble)
                objective += scaledPenalty * numpy.sum(numpy.square(itemBiases), dtype = numpy.longdouble)
                objective += scaledPenalty * globalBias * globalBias
            
        gradient = Mat2Vec(userVGradient, itemVGradient, userBGradient, itemBGradient, globalBGradient)

        if verbose:
            print( ".",)
            sys.stdout.flush()
        
        return objective, gradient
    
    def ObjectiveOnly(paramVector):
        objective, gradient = Objective(paramVector)
        return objective
    def GradientOnly(paramVector):
        objective, gradient = Objective(paramVector)
        return gradient
    
    userVectorsInit = None
    itemVectorsInit = None
    userBiasesInit = None
    itemBiasesInit = None
    globalBiasInit = None
    if start_vec is None:
        userVectorsInit = numpy.random.standard_normal((numUsers, num_dimensions))
        itemVectorsInit = numpy.random.standard_normal((numItems, num_dimensions))
        userBiasesInit = numpy.zeros(numUsers, dtype = numpy.float)
        itemBiasesInit = numpy.zeros(numItems, dtype = numpy.float)
        globalBiasInit = 0
    else:
        userVectorsInit = start_vec[0]
        itemVectorsInit = start_vec[1]
        userBiasesInit = start_vec[2]
        itemBiasesInit = start_vec[3]
        globalBiasInit = start_vec[4]
    
    startVector = Mat2Vec(userVectorsInit, itemVectorsInit, userBiasesInit, itemBiasesInit, globalBiasInit)

    if verbose:
        print("MF.GENERATE_MATRIX: [DBG]\t Checking gradients")
        print(scipy.optimize.check_grad(ObjectiveOnly, GradientOnly, startVector))

    ops = {'maxiter': 2000, 'disp': False, 'gtol': 1e-5,\
            'ftol': 1e-5, 'maxcor': 50}

    result = scipy.optimize.minimize(fun = Objective, x0 = startVector,
                    method = 'L-BFGS-B', jac = True, tol = 1e-5, options = ops)
    
    if verbose:
        print("")
        print("MF.GENERATE_MATRIX: [DBG]\t Optimization result:", result['message'])
        sys.stdout.flush()

    return Vec2Mat(result['x'])

In [ ]:
if __name__ == "__main__":
    import scipy.sparse
    
    rows = [2,1,4,3,0,4,3]
    cols = [0,2,1,1,0,0,0]
    vals = [1,2,3,4,5,4,5]
    checkY = scipy.sparse.coo_matrix((vals, (rows,cols)), dtype = numpy.int)
    checkY = checkY.toarray()
    checkY = numpy.ma.array(checkY, dtype = numpy.int, mask = checkY <= 0, hard_mask = True, copy = False)
    print("[MAIN]\t Partially observed ratings matrix")
    print(checkY)

    randomPropensities = numpy.random.random(size = numpy.shape(checkY))
    randomInvPropensities = numpy.reciprocal(randomPropensities)

    userVectors, itemVectors, userBiases, itemBiases, globalBias = GENERATE_MATRIX(checkY, None, 1.0, 5, 'Vanilla',
                                                'Regularized', 'MSE', None, verbose = True)

    userVectors, itemVectors, userBiases, itemBiases, globalBias = GENERATE_MATRIX(checkY, randomInvPropensities, 1.0, 5, 'Vanilla',
                                                'Regularized', 'MSE', None, verbose = True)

    userVectors, itemVectors, userBiases, itemBiases, globalBias = GENERATE_MATRIX(checkY, randomInvPropensities, 1.0, 5, 'Vanilla',
                                                'Regularized', 'MAE', None, verbose = True)

    userVectors, itemVectors, userBiases, itemBiases, globalBias = GENERATE_MATRIX(checkY, randomInvPropensities, 1.0, 5, 'SelfNormalized',
                                                'Regularized', 'MSE', None, verbose = True)

    userVectors, itemVectors, userBiases, itemBiases, globalBias = GENERATE_MATRIX(checkY, None, 1.0, 5, 'Vanilla',
                                                'Free', 'MSE', None, verbose = True)

    userVectors, itemVectors, userBiases, itemBiases, globalBias = GENERATE_MATRIX(checkY, randomInvPropensities, 1.0, 5, 'SelfNormalized',
                                                'None', 'MSE', None, verbose = True)


    print("[MAIN]\t User vectors")
    print(userVectors)
    print("[MAIN]\t Item vectors")
    print(itemVectors)
    print("[MAIN]\t User biases")
    print(userBiases)
    print("[MAIN]\t Item biases")
    print(itemBiases)
    print("[MAIN]\t Global bias")
    print(globalBias)
    
    completeScores = PREDICTED_SCORES(userVectors, itemVectors, userBiases, itemBiases, globalBias, True)
    print("[MAIN]\t Predicted scores")
    print(completeScores)

## Expt2

In [ ]:
# import MF
# import numpy
import scipy.sparse.linalg
# import sys

In [ ]:
def MF_TRAIN(params, train_observations, inv_propensities, normalization, metric, start_vector):
    retVal = None
    actualStart = None
    if start_vector is not None:
        actualStart = (start_vector[0][:,0:params[1]], start_vector[1][:,0:params[1]],
                        start_vector[2], start_vector[3], start_vector[4])

    tempInvPropensities = None
    if inv_propensities is not None:
        tempInvPropensities = (4.0 / 3.0) * inv_propensities
        if params[2] >= 0:
            tempInvPropensities = numpy.clip(tempInvPropensities, a_min = 0, a_max = params[2])

    retVal = MF.GENERATE_MATRIX(train_observations, tempInvPropensities, params[0], 
                                params[1], normalization, bias_mode = params[3], mode = metric, 
                                start_vec = actualStart, verbose = False)

    return retVal

In [ ]:
def FINAL_TRAIN(approach_tuple, metric, observations, start_vector):
    invP = approach_tuple[1]
    normN = approach_tuple[2]
    bestLambda = approach_tuple[3][0]
    bestDims = approach_tuple[3][1]
    bestClip = approach_tuple[3][2]
    bestBias = approach_tuple[3][3]
    actualStart = None
    if start_vector is not None:
        actualStart = (start_vector[0][:,0:bestDims], start_vector[1][:,0:bestDims],
                        start_vector[2], start_vector[3], start_vector[4])

    tempInvP = None
    if bestClip < 0 or invP is None:
        tempInvP = invP
    else:
        tempInvP = numpy.clip(invP, a_min = 0, a_max = bestClip)

    retVal = MF.GENERATE_MATRIX(observations, tempInvP, bestLambda, bestDims, normN, bias_mode = bestBias,
                        mode = metric, start_vec = actualStart)

    return retVal

In [ ]:
def INIT_PARAMS(partial_observations, num_dimensions):
    averageObservedRating = numpy.ma.mean(partial_observations, dtype = numpy.longdouble)
    completeRatings = numpy.ma.filled(partial_observations.astype(numpy.float), averageObservedRating)
    numUsers, numItems = numpy.shape(partial_observations)

    u,s,vt = scipy.sparse.linalg.svds(completeRatings, k = num_dimensions, ncv = 50, tol = 1e-7, which = 'LM', 
                        v0 = None, maxiter = 2000, return_singular_vectors = True)
            
    startTuple = (u, numpy.transpose(numpy.multiply(vt, s[:,None])), 
                     numpy.zeros(numUsers, dtype = numpy.longdouble), 
                     numpy.zeros(numItems, dtype = numpy.longdouble), 
                     averageObservedRating)
    return startTuple 

In [ ]:
def TRAIN_HELPER(approach, gold_inv_propensities, nb_inv_propensities):
    invP = None
    if approach == 'Naive':
        invP = None
    elif approach.startswith('Gold'):
        invP = gold_inv_propensities
    elif approach.startswith('NB'):
        invP = nb_inv_propensities
    else:
        print ("TRAIN_HELPER: [ERR] Unrecognized approach", approach)
        sys.exit(0)

    normN = None
    if approach == 'Naive' or approach.endswith('-IPS'):
        normN = 'Vanilla'
    elif approach.endswith('-SNIPS'):
        normN = 'SelfNormalized'
    elif approach.endswith('-UNIPS'):
        normN = 'UserNormalized'
    elif approach.endswith('-INIPS'):
        normN = 'ItemNormalized'
    else:
        print ("TRAIN_HELPER: [ERR] Unrecognized approach", approach)
        sys.exit(0)
        
    return invP, normN

In [ ]:
if __name__ == "__main__":
    import argparse
#     import Datasets
    from lenskit.datasets import ML100K
#     import Metrics
    import Propensity
    import pickle
    import os
    import itertools
    from joblib import Parallel, delayed
    
    parser = argparse.ArgumentParser(description='Semi-Synthetic Learning on ML100K.')
    parser.add_argument('--seed', '-s', metavar='S', type=int, 
                        help='Seed for numpy.random', default=387)
    parser.add_argument('--trial', '-t', metavar='T', type=int, 
                        help='Trial ID', default=1)
    parser.add_argument('--alphas', '-a', metavar='A', type=str, 
                        help='Alpha values', default='1,0.5,0.25,0.125,0.0625,0.03125')
    parser.add_argument('--lambdas', '-l', metavar='L', type=str, 
                        help='Lambda values', default='0.008,0.04,0.2,1,5,25,125')
    parser.add_argument('--numdims', '-n', metavar='N', type=str, 
                        help='Dimension values', default='20')
    parser.add_argument('--clips', '-c', metavar='C', type=str, 
                        help='Clip values', default='-1')
    parser.add_argument('--estimators', '-e', metavar='E', type=str, 
                        help='Learning methods', default='Naive,Gold-IPS,Gold-SNIPS')
    parser.add_argument('--metric', '-m', metavar='M', type=str, 
                        help='Metrics', default='MSE')
                        
    args = parser.parse_args()
    numpy.random.seed(args.seed)
    
    approaches = args.estimators.strip().split(',')
    
    approachDict = {}
    for approach in approaches:
        approachDict[(approach, args.metric)] = len(approachDict)
        
    alphas = []
    tokens = args.alphas.strip().split(',')
    for token in tokens:
        alphas.append(float(token))
    
    lambdas = []
    tokens = args.lambdas.strip().split(',')
    for token in tokens:
        lambdas.append(float(token))
        
    numDims = []
    tokens = args.numdims.strip().split(',')
    for token in tokens:
        numDims.append(int(token))
    
    clipVals = []
    tokens = args.clips.strip().split(',')
    for token in tokens:
        clipVals.append(int(token))

    numAlphas = len(alphas)
    numApproaches = len(approachDict)
    
    biasModes = ['Free']
    numDimSettings = len(numDims)
    numClipSettings = len(clipVals)
    numBiasModes = len(biasModes)
 
    numLambdas = len(lambdas)
    numParamSettings = numLambdas * numDimSettings * numClipSettings * numBiasModes
    
    paramSettings = list(itertools.product(lambdas, numDims, clipVals, biasModes))
       
#     ML100KCompleteTest = Datasets.ML100K('../')
    ML100KCompleteTest = ML100K('../')
    
    allEstimates = numpy.zeros((numApproaches, numAlphas), dtype = numpy.longdouble)
    
    currMetric = None
    if args.metric == 'MSE':
        currMetric = Metrics.MSE
    elif args.metric == 'MAE':
        currMetric = Metrics.MAE
    else:
        print("Expt2: [ERR] Unrecognized metric", args.metric)
        sys.exit(0)
        
    print ("Expt2: [LOG] Starting metric", args.metric)
        
    def updateResults(val, approach, ind):
        approachTuple = (approach, args.metric)
        approachIndex = approachDict[approachTuple]
        allEstimates[approachIndex, ind] = val
            
    print ("Expt2: [LOG] Trial", args.trial)
    numpy.random.seed(args.seed + args.trial)
    
    outputFile = '../logs/expt2/'+str(args.seed)+'_'+str(args.trial)+'_'+args.metric+'_'
    
    for ind, alpha in enumerate(alphas):
        print ("Expt2: [LOG] Alpha:", alpha)

        partialObservations, goldPropensities = Propensity.PARTIAL_OBSERVE(ML100KCompleteTest, alpha, 0.05, verbose = False)
       
        flatObservations = numpy.ma.compressed(partialObservations) 
        observedHistogram = numpy.bincount(flatObservations, minlength = 6)[1:]
        observedHistogram = observedHistogram.astype(numpy.longdouble) / \
                                observedHistogram.sum(dtype = numpy.longdouble)
        print ("Expt2: [LOG] Observed Marginals: ", observedHistogram)
       
        goldInvPropensities = numpy.reciprocal(goldPropensities)
        
        foldScores = numpy.zeros((numApproaches, 4, numParamSettings), dtype = numpy.longdouble)
        foldTestScores = numpy.zeros((numApproaches, 4, numParamSettings), dtype = numpy.longdouble)
        
        observationIndices = numpy.ma.nonzero(partialObservations)
        numObservations = numpy.ma.count(partialObservations)
 
        shuffleIndices = numpy.random.permutation(numObservations)
        fractionObservations = int(numObservations/4)
        firstFold = shuffleIndices[:fractionObservations]
        secondFold = shuffleIndices[fractionObservations:2*fractionObservations]
        thirdFold = shuffleIndices[2*fractionObservations:3*fractionObservations]
        fourthFold = shuffleIndices[3*fractionObservations:]
        print ("Expt2: [LOG] Split %d observations into folds. Fold sizes:" % len(shuffleIndices),\
                        len(firstFold), len(secondFold), len(thirdFold), len(fourthFold))
        
        for fold in xrange(4):
            print ("Expt2: [LOG] Fold:", fold)
            trainObservations = numpy.ma.copy(partialObservations)
            testObservations = numpy.ma.copy(partialObservations)

            if fold == 0:
                trainObservations[observationIndices[0][firstFold], observationIndices[1][firstFold]] = \
                                        numpy.ma.masked

                testObservations[observationIndices[0][secondFold], observationIndices[1][secondFold]] = \
                                        numpy.ma.masked
                testObservations[observationIndices[0][thirdFold], observationIndices[1][thirdFold]] = \
                                        numpy.ma.masked
                testObservations[observationIndices[0][fourthFold], observationIndices[1][fourthFold]] = \
                                        numpy.ma.masked
            elif fold == 1:
                trainObservations[observationIndices[0][secondFold], observationIndices[1][secondFold]] = \
                                        numpy.ma.masked

                testObservations[observationIndices[0][firstFold], observationIndices[1][firstFold]] = \
                                        numpy.ma.masked
                testObservations[observationIndices[0][thirdFold], observationIndices[1][thirdFold]] = \
                                        numpy.ma.masked
                testObservations[observationIndices[0][fourthFold], observationIndices[1][fourthFold]] = \
                                        numpy.ma.masked
            elif fold == 2:
                trainObservations[observationIndices[0][thirdFold], observationIndices[1][thirdFold]] = \
                                        numpy.ma.masked

                testObservations[observationIndices[0][firstFold], observationIndices[1][firstFold]] = \
                                        numpy.ma.masked
                testObservations[observationIndices[0][secondFold], observationIndices[1][secondFold]] = \
                                        numpy.ma.masked
                testObservations[observationIndices[0][fourthFold], observationIndices[1][fourthFold]] = \
                                        numpy.ma.masked
            elif fold == 3:
                trainObservations[observationIndices[0][fourthFold], observationIndices[1][fourthFold]] = \
                                        numpy.ma.masked

                testObservations[observationIndices[0][firstFold], observationIndices[1][firstFold]] = \
                                        numpy.ma.masked
                testObservations[observationIndices[0][secondFold], observationIndices[1][secondFold]] = \
                                        numpy.ma.masked
                testObservations[observationIndices[0][thirdFold], observationIndices[1][thirdFold]] = \
                                        numpy.ma.masked
            else:
                print ("Expt2: [ERR] #Folds not supported ", fold)
                sys.exit(0)
                   
            #Get starting params by SVD
            startFileName = outputFile + 'fold' + str(fold) + '_' + str(alpha) +'_init.pkl'
            startTuple = None
            if os.path.exists(startFileName):
                g = open(startFileName, 'rb')
                startTuple = pickle.load(g)
                g.close()
            else:
                startTuple = INIT_PARAMS(trainObservations, 20)
                g = open(startFileName, 'wb')
                pickle.dump(startTuple, g, -1)
                g.close()
            
            for approach in approaches:
                print ("Starting approach ", approach)
                invP, normN = TRAIN_HELPER(approach, goldInvPropensities, None)
                approachTuple = (approach, args.metric)
                approachIndex = approachDict[approachTuple]
                modelFileName = outputFile + 'fold' + str(fold) + '_' + str(alpha) +'_'+approach+'.pkl'
                modelsPerLambda = None
                if os.path.exists(modelFileName):
                    g = open(modelFileName, 'rb')
                    modelsPerLambda = pickle.load(g)
                    g.close()
                    print ("Expt2: [LOG]\t Loaded trained models for each lambda from ", modelFileName)
                else:
                    modelsPerLambda = Parallel(n_jobs = -1, verbose = 0)(delayed(MF_TRAIN)(l2Lambda, 
                                            trainObservations, invP, normN, args.metric, startTuple)
                                            for l2Lambda in paramSettings)
                    g = open(modelFileName, 'wb')
                    pickle.dump(modelsPerLambda, g, -1)
                    g.close()
                    print ("Expt2: [LOG]\t Saved trained models for each lambda to ", modelFileName)

                for lambdaIndex, eachModel in enumerate(modelsPerLambda):
                    selectedBiasMode = paramSettings[lambdaIndex][3]
                    selectedBias = True
                    if selectedBiasMode == 'None':
                        selectedBias = False

                    predictedY = MF.PREDICTED_SCORES(eachModel[0], eachModel[1], 
                                                eachModel[2], eachModel[3], eachModel[4], use_bias = selectedBias)
                    score = None
                    if invP is not None:
                        score = currMetric(testObservations, predictedY, 4.0*invP)
                    else:
                        score = currMetric(testObservations, predictedY, None)
                        
                    if normN == 'Vanilla':
                        score = score[0]
                    elif normN == 'SelfNormalized':
                        score = score[1]
                    elif normN == 'UserNormalized':
                        score = score[2]
                    elif normN == 'ItemNormalized':
                        score = score[3]
                    else:
                        print ("Expt2: [ERR] Normalization not supported for metric ", normN, args.metric)
                        sys.exit(0)
                    
                    foldScores[approachIndex, fold, lambdaIndex] = score
                    
                    foldTestScore = currMetric(ML100KCompleteTest, predictedY, None)[0]
                    foldTestScores[approachIndex, fold, lambdaIndex] = foldTestScore
                    print ("Expt2: [LOG] Lambda/NumDims: ", paramSettings[lambdaIndex],
                                "\t Test Fold Score: ", score, "\t Test Set Score: ", foldTestScore)
                
                #Save foldScores and foldTestScores after each approach in each fold. Overwrite if needed.
                scoresFile = outputFile + 'Alpha'+ str(alpha)+'_foldScores.pkl'
                scoresData = (foldScores, foldTestScores)
                g = open(scoresFile, 'wb')
                pickle.dump(scoresData, g, -1)
                g.close()
                sys.stdout.flush()        
        
        eventualApproachParams = []
        for approach in approaches:
            invP, normN = TRAIN_HELPER(approach, goldInvPropensities, None)
            approachTuple = (approach, args.metric)
            approachIndex = approachDict[approachTuple]
            approachScores = foldScores[approachIndex,:,:]
            allFoldScores = approachScores.sum(axis = 0, dtype = numpy.longdouble)
            bestLambdaIndex = numpy.argmin(allFoldScores)
            bestLambda = paramSettings[bestLambdaIndex]
            print ("FINAL_TRAIN: [LOG] Retraining ", approach, " Best lambda/numDims", bestLambda    )
            for everyLambdaIndex, everyLambda in enumerate(paramSettings):
                print("FINAL_TRAIN: [DBG] AllFoldScores: ", approach, everyLambda, allFoldScores[everyLambdaIndex])
            eventualApproachParams.append((approach,invP,normN,bestLambda))
            
        finalModels = None
        finalModelFileName = outputFile + str(alpha) +'_finalmodels.pkl'
        if os.path.exists(finalModelFileName):
            g = open(finalModelFileName, 'rb')
            finalModels = pickle.load(g)
            g.close()
            print ("Expt2: [LOG]\t Loaded trained final models from ", finalModelFileName)
        else:
            finalModels = Parallel(n_jobs = -1, verbose = 0)(delayed(FINAL_TRAIN)(approachTup, args.metric, 
                                        partialObservations, startTuple)
                                        for approachTup in eventualApproachParams)
            g = open(finalModelFileName, 'wb')
            pickle.dump(finalModels, g, -1)
            g.close()
            print ("Expt2: [LOG]\t Saved trained final models to ", finalModelFileName)
        
        for approachID, approachTuple in enumerate(eventualApproachParams):
            resultTuple = finalModels[approachID]
            finalBiasMode = approachTuple[3][3]
            finalBias = True
            if finalBiasMode == 'None':
                finalBias = False
            predictedY = MF.PREDICTED_SCORES(resultTuple[0], resultTuple[1], \
                                resultTuple[2], resultTuple[3], resultTuple[4], use_bias = finalBias)

            metricValue = currMetric(ML100KCompleteTest, predictedY, None)[0]
            print ("Expt2: [LOG] ", approachTuple[0], "\t Eventual result:", metricValue)
            sys.stdout.flush()
            updateResults(metricValue, approachTuple[0], ind)    
        
        #Dump results after each alpha. Overwrite if needed.
        outputData = (approaches, approachDict, alphas, allEstimates)
        g = open(outputFile + args.estimators +'.pkl', 'wb')
        pickle.dump(outputData, g, -1)
        g.close()

## Expt3

## Train

## Test